In [1]:
import pandas as pd
import numpy as np
import os, sys
from datetime import datetime
import preprocessor as p
from gensim.parsing.preprocessing import remove_stopwords
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
nltk.download
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gov_m\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
%%time

# Read the tweet file to a dataframe 
tweets = pd.read_csv('tweets_00.csv', low_memory=False)

Wall time: 30.2 s


In [3]:
# Pre-Check
tweets.head()

,Unnamed: 0,coordinates,id,hashtags,place,text,user_location
0,0,NaN,1028954403129184256,NaN,NaN,Gotta love the facts. https://t.co/bZ2G8AZuo9,United States
1,1,NaN,1028954356572250112,NaN,NaN,RT @ToolangiForest: A great day of action for ...,Australia
2,2,NaN,1028954497341480960,NaN,NaN,@jonkudelka Harvey Norman reckons climate chan...,"Noble Park,Oz, Khon Kaen Thai"
3,3,NaN,1028954494133043200,NaN,NaN,RT @jayrosen_nyu: Why does skepticism about im...,"Toronto, Ontario"
4,4,NaN,1028954811511844864,climatechange,NaN,RT @FranceinIreland: On 5th November we call a...,"Dublin, Ireland"


In [4]:
# Pre-Check
tweets.count()

Unnamed: 0       7194837
coordinates         1955
id               7194837
hashtags         1760731
place              83115
text             7194837
user_location    5431846
dtype: int64

In [5]:
%%time
# Drop Unnamed Columns
tweets = tweets.drop(columns='Unnamed: 0')
# tweets = tweets.drop(columns='Unnamed: 0.1')

Wall time: 448 ms


In [6]:
# Pre-Process Function
def preprocess_tweet(row):
    # Function for cleaning text - removal of URLs, Mentions
    # Using tweet-preprocessor package
    text = p.clean(row['text'])
    
    # Lowercase
    text = text.lower()
    
    # Function for Punctuation Removal, Replace extra white spaces, stopwords removal
    # Using gensim package
    text = remove_stopwords(text)
    
    # Remove extra white spaces
    text = text.replace('[^\w\s]',' ').replace('\s\s+', ' ')
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove Digits
    text = text.replace('\d+', '')
    
    # Lemmatization + Tokenization
    lemmatizer = nltk.stem.WordNetLemmatizer()
    w_tokenizer = TweetTokenizer()
    
    text = [(lemmatizer.lemmatize(w)) for w in w_tokenizer.tokenize((text))]
    
    return text

In [7]:
%%time

# Run Preprocessing
tweets['text'] = tweets.apply(preprocess_tweet, axis=1)

Wall time: 55min 26s


In [8]:
# View
tweets.head()

,coordinates,id,hashtags,place,text,user_location
0,NaN,1028954403129184256,NaN,NaN,"[gotta, love, fact]",United States
1,NaN,1028954356572250112,NaN,NaN,"[great, day, action, message, dear, dan, toola...",Australia
2,NaN,1028954497341480960,NaN,NaN,"[harvey, norman, reckons, climate, change, bun...","Noble Park,Oz, Khon Kaen Thai"
3,NaN,1028954494133043200,NaN,NaN,"[skepticism, immigration, walk, hand, hand, sk...","Toronto, Ontario"
4,NaN,1028954811511844864,climatechange,NaN,"[th, november, creative, citizen, w, practical...","Dublin, Ireland"


In [9]:
tweets.to_pickle('tweets_00_Pickle.pkl')